In [27]:
#Task 2

import numpy as np
import pandas as pd
import scipy.stats as stats

# Simulation parameters
n_simulations = 10**4  # Number of simulations to approximate the distribution
beta = 10**8  # Inverse temperature
lambda_step = 10**(-4)  # Step size for SGLD
gamma = 10**(-8)  # Regularization parameter
theta = 0  # Initial guess for theta

# Set quantile levels and number of iterations for SGLD
q = 0.95
n_iterations = 10**6
n_assets = 3  # Number of assets in the portfolio
# Define the initial weights for the portfolio
initial_weights = np.array([1.0 / n_assets] * n_assets)

# Asset parameters for each scenario

asset_params = [
    [{'mu': 500, 'sigma': 1}, {'mu': 0, 'sigma': 10**6}, {'mu': 0, 'sigma': 10**-4}],
    [{'mu': 500, 'sigma': 1}, {'mu': 0, 'sigma': 10**6}, {'mu': 0, 'sigma': 1}],
    [{'mu': 0, 'sigma': 10**3}, {'mu': 0, 'sigma': 1}, {'mu': 0, 'sigma': 4}],
    [{'mu': 0, 'sigma': 1}, {'mu': 1, 'sigma': 4}, {'mu': 0, 'sigma': 10**-4}],
    [{'mu': 0, 'sigma': 1}, {'mu': 1, 'sigma': 4}, {'mu': 2, 'sigma': 1}]
]


In [30]:
# Simulate asset returns
def simulate_asset_returns(params, size):
        return np.random.normal(params['mu'], params['sigma'], size)

# Calculate VaR and CVaR from simulated returns
def calculate_var_cvar(returns, q):
    var = np.quantile(returns, q)
    # Sort returns
    sorted_returns = np.sort(returns)
    # Find the index where VaR would be positioned in the sorted list
    var_index = np.searchsorted(sorted_returns, var, side='right')
    # CVaR is the average of the worst 1-q percent of returns
    cvar = np.mean(sorted_returns[var_index:])
    return var, cvar

def g_i(weights, i):
    """
    Compute the softmax of the i-th element of the weights vector.

    :param weights: The vector of portfolio weights.
    :param i: The index of the weight to apply the softmax function.
    :return: The softmax of the i-th weight.
    """
    exp_weights = np.exp(weights)
    softmax_i = exp_weights[i] / np.sum(exp_weights)
    return softmax_i

def partial_g_w(weights, asset_returns, j):
    sum_exp_weights = np.sum(np.exp(weights))
    g_w = np.zeros_like(weights)
    for i in range(len(weights)):
        if i != j:
            g_w[i] = np.sum(-asset_returns[i] * (np.exp(weights[i]) * np.exp(weights[j])) / sum_exp_weights**2)
        else:
            g_w[i] = np.sum(asset_returns[i] * (sum_exp_weights - np.exp(weights[i])) / sum_exp_weights**2)
    return np.sum(g_w)

def H_theta(theta_hat, weights, asset_returns, q, gamma):
    # portfolio_loss = sum(weight * returns - theta_hat for weight, returns in zip(weights, asset_returns))
    # positive_losses = portfolio_loss > 0
    # return np.mean(np.maximum(0, positive_losses) * (1 / (1 - q)) + theta_hat) + gamma * theta_hat**2
    # Compute the portfolio loss for each simulation
    portfolio_losses = ((np.dot(np.exp(weights), asset_returns))/ (np.sum(np.exp(weights)))) - theta_hat
    # Filter for positive losses
    positive_losses = portfolio_losses > 0
    # Calculate the average loss where the condition is True
    loss = np.mean(portfolio_losses[positive_losses]) if np.any(positive_losses) else 0
    # Regularization term
    regularization = 2 * gamma * theta_hat**2
    # The complete loss is the sum of the two terms
    return 1/(1 - q) * loss + regularization

def H_wj(theta_hat, weights, j, asset_returns, q, gamma):
    # portfolio_loss = sum(weight * returns - theta_hat for weight, returns in zip(weights, asset_returns))
    # positive_losses = portfolio_loss > 0
    # return 1/(1 - q) * partial_g_w(weights, asset_returns, j) * positive_losses + 2 * gamma * weights[j]
    # Compute the portfolio loss for each simulation
    portfolio_losses = ((np.dot(np.exp(weights), asset_returns))/ (np.sum(np.exp(weights)))) - theta_hat
    # Filter for positive losses
    positive_losses = portfolio_losses > 0
    # Calculate the average loss where the condition is True
    loss = np.mean(portfolio_losses[positive_losses]) if np.any(positive_losses) else 0
    return 1/(1 - q) * partial_g_w(weights, asset_returns, j) * loss + 2 * gamma * weights[j]

In [31]:
# The main SGLD optimization routine
def sgld_optimization(asset_params, q, gamma, lambda_step, n_iterations, beta):
    # Initialize theta and weights
    current_theta = 0
    weights = initial_weights
    
    for _ in range(n_iterations):
        asset_returns = np.array([simulate_asset_returns(params, n_simulations) for params in asset_params])
        portfolio_returns = np.dot(weights, asset_returns)
        # Calculate the gradient for theta and update it
        theta_gradient = H_theta(current_theta, weights, asset_returns, q, gamma)
        current_theta -= lambda_step * theta_gradient + np.sqrt(2 * lambda_step / beta) * np.random.randn()

        # Calculate the gradient for each weight and update them
        for j in range(len(weights)):
            wj_gradient = H_wj(current_theta, weights, j, asset_returns, q, gamma)
            weights[j] -= lambda_step * wj_gradient + np.sqrt(2 * lambda_step / beta) * np.random.randn()

        
    # Calculate the portfolio's VaR and CVaR using the optimized weights and theta
    VaR, CVaR = calculate_var_cvar(portfolio_returns, q)
    
    return weights, current_theta, VaR, CVaR

# Run the optimization for one of the scenarios
optimized_weights, optimized_theta, VaR, CVaR = sgld_optimization(
    asset_params=asset_params[0],  # Use the first scenario for this example
    q=q,
    gamma=gamma,
    lambda_step=lambda_step,
    n_iterations=n_iterations,
    beta=beta
)

# Print the optimized weights and risk measures
print("Optimized weights:", optimized_weights)
print("Optimized theta (VaR):", optimized_theta)
print("VaR:", VaR)
print("CVaR:", CVaR)

In [ ]:
# SGLD optimization process
def sgld_optimization(asset_params, initial_weights, q, gamma, lambda_step, n_iterations, beta):
    #full = []
    current_theta = 0
    weights = np.copy(initial_weights)
    #for s in scene:
    for _ in range(n_iterations):
        
        # Simulate asset returns for each asset
        asset_returns = simulate_asset_returns(asset_params, n_simulations) 
        
        # Update theta_hat using the SGLD step for theta
        theta_gradient = H_theta(current_theta, asset_returns, weights, q, gamma)
        theta_hat = current_theta - lambda_step * theta_gradient + np.sqrt(2 * lambda_step / beta) * np.random.randn()
        
        # Update each weight using the SGLD step for weights
        new_weights = np.zeros_like(weights)
        for j in range(len(weights)):
            wj_gradient = H_wj(current_theta, asset_returns, weights, j, q, gamma)
            new_weights[j] = weights[j] - lambda_step * wj_gradient + np.sqrt(2 * lambda_step / beta) * np.random.randn()
        
        # Ensure weights sum to 1 (if this is a constraint in your portfolio)
        new_weights /= np.sum(new_weights)
        
        weights = new_weights
        current_theta = theta_hat
        #full.append(weights, current_theta)
    return weights, current_theta

# Run the optimization with given parameters
# Note: You need to define 'asset_params' with your actual asset parameters before running this.
theta_hat, optimized_weights = sgld_optimization(
    asset_params, initial_weights, q, gamma, lambda_step, n_iterations, beta
)

# Output the optimized theta and weights
print(f"Optimized theta: {theta_hat}")
print(f"Optimized weights: {optimized_weights}")

TypeError: list indices must be integers or slices, not str

In [4]:
# SGLD update step for theta and weights
def sgld_update(current_theta, asset_returns, weights, gamma, q, lambda_step):
    # Update theta
    theta_gradient = H_theta(current_theta, asset_returns, weights, q, gamma)
    theta_noise = np.sqrt(2 * lambda_step / beta) * np.random.randn()
    theta_hat = current_theta - lambda_step * theta_gradient + theta_noise
    
    # Update each weight
    for j in range(len(weights)):
        wj_gradient = H_wj(current_theta, asset_returns, weights, j, q, gamma)
        wj_noise = np.sqrt(2 * lambda_step / beta) * np.random.randn()
        weights[j] = weights[j] - lambda_step * wj_gradient + wj_noise
        
    return theta_hat, weights

# SGLD optimization process to find the optimal theta and weights
def sgld_optimization(asset_params, initial_weights, q, gamma, lambda_step, n_iterations):
    theta_hat = 0
    weights = np.copy(initial_weights)
    
    for iteration in range(n_iterations):
        # Simulate returns for each asset for the current iteration
        asset_returns = np.array(simulate_asset_returns(asset_params) )
        # Update theta and weights using the SGLD step
        theta_hat, weights = sgld_update(theta_hat, asset_returns, weights, gamma, q, lambda_step)
        
    # Calculate the final VaR and CVaR using the optimized theta and weights
    asset_returns = simulate_asset_returns(asset_params)
    portfolio_returns = np.sum([w * r for w, r in zip(weights, asset_returns)], axis=0)
    var, cvar = calculate_var_cvar(portfolio_returns, q)
    
    return theta_hat, weights, var, cvar

theta_hat, optimized_weights, var, cvar = sgld_optimization(
    asset_params, initial_weights, q, gamma, lambda_step, n_iterations
)

theta_hat, optimized_weights, var, cvar

NameError: name 'simulate_returns_for_scenario' is not defined

In [ ]:
# SGLD update step
def sgld_update(current_theta, asset_returns, weights, gamma, lambda_step,j):
    portfolio_loss = sum(weights[i] * asset_returns[i] - current_theta for i in range(len(asset_returns)))
    positive_losses = portfolio_loss > 0
    # Implementation of the provided loss function derivative
    
    return current_theta - lambda_step * (H_theta(current_theta, weights, asset_returns, q, gamma) + H_wj(current_theta,  weights, j, asset_returns, q, gamma))+ np.sqrt(2 * lambda_step / beta) * np.random.randn()

# SGLD optimization process
def sgld_optimization(asset_params, q, gamma, lambda_step, n_iterations):
    # Initialize theta
    current_theta = 0
    for iteration in range(n_iterations):
        # Simulate returns for each asset for the current iteration
        asset_returns = [simulate_asset_returns(params['mu'], params['sigma'], 1) for params in asset_params]
        # Update theta using the SGLD step
        current_theta = sgld_update(current_theta, asset_returns, weights, gamma, lambda_step)
    return current_theta


In [ ]:
# Initial weights and returns simulation for all assets
weights = np.array([1/n_assets] * n_assets)  # Equal weights for simplicity
asset_returns = [simulate_asset_returns(params['mu'], params['sigma'], n_simulations) for params in asset_params]

# Aggregate portfolio returns
portfolio_returns = np.sum([w * r for w, r in zip(weights, asset_returns)], axis=0)

# Run SGLD optimization to minimize the loss function
optimized_theta = sgld_optimization(asset_params, q, gamma, lambda_step, n_iterations)

# Calculate the VaR and CVaR based on the optimized theta
var, cvar = calculate_var_cvar(portfolio_returns - optimized_theta, q)

optimized_theta, var, cvar

TypeError: list indices must be integers or slices, not str